In [1]:
import numpy as np
import pandas as pd
import time, datetime
import sys, os, os.path
import scipy, scipy.signal

from datetime import date, datetime

from sklearn.linear_model import LinearRegression
from sklearn.metrics import f1_score, accuracy_score, \
                            confusion_matrix, balanced_accuracy_score, \
                            classification_report
import matplotlib.pyplot as plt

# from patsy import cr
# from pprint import pprint
# from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [2]:
sys.path.append('/Users/hn/Documents/00_GitHub/Ag/NASA/Python_codes/')
import NASA_core as nc
import NASA_plot_core as ncp

/Library/Python/3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
NASA_dir_base = "/Users/hn/Documents/01_research_data/NASA/"
train_TS_dir_base = NASA_dir_base + "VI_TS/"
training_set_dir = NASA_dir_base + "/ML_data_Oct17/"

meta_dir = NASA_dir_base + "parameters/"
data_part_of_shapefile_dir = NASA_dir_base + "data_part_of_shapefile/"

In [4]:
# MacBook directories

NASA_dir_base = "/Users/hn/Documents/01_research_data/Amin/inclusion_prob/"
data_part_of_shapefile_dir = NASA_dir_base + "data_part_of_shapefile/"
training_set_dir = NASA_dir_base

In [5]:

# meta = pd.read_csv(meta_dir+"evaluation_set.csv")
# meta_moreThan10Acr=meta[meta.ExctAcr > 10]
# print (meta.shape)
# print (meta_moreThan10Acr.shape)
# meta.head(2)

In [6]:
ground_truth_labels = pd.read_csv(training_set_dir+"groundTruth_labels_Oct17_2022.csv")
print ("Unique Votes: ", ground_truth_labels.Vote.unique())
print (len(ground_truth_labels.ID.unique()))
ground_truth_labels.head(2)

Unique Votes:  [2 1]
3160


,ID,Vote
0,99837_WSDA_SF_2017,2
1,114615_WSDA_SF_2017,1


In [7]:
csv_data_parts = os.listdir(data_part_of_shapefile_dir)
csv_data_parts = [x for x in csv_data_parts if x.endswith("csv")]
csv_data_parts

['FranklinYakima2018.csv',
 'irriigated_SF_data_concatenated.csv',
 'all_eastern_centroid.csv',
 'Eastern_2018_centroid.csv',
 'Eastern_2015_centroid.csv',
 'Walla2015.csv',
 'AdamBenton2016.csv',
 'all_SF_data_concatenated.csv',
 'Eastern_2016_centroid.csv',
 'Eastern_2017_centroid.csv',
 'Monterey2014.csv',
 'Grant2017.csv']

In [8]:
irriigated_SF_data = pd.read_csv(data_part_of_shapefile_dir + "irriigated_SF_data_concatenated.csv")
irriigated_SF_data.head(2)

,ID,CropTyp,Acres,ExctAcr,Irrigtn,LstSrvD,DataSrc,county
0,51380_WSDA_SF_2016,bluegrass seed,6,6.116621,sprinkler,2016/09/19 00:00:00,wsda,Adams
1,51381_WSDA_SF_2016,wheat,117,116.743101,center pivot,2016/09/20 00:00:00,wsda,Adams


In [9]:
irriigated_SF_data.county.unique()

array(['Adams', 'Benton', 'Franklin', 'Yakima', 'Grant', 'Walla Walla'],
      dtype=object)

In [10]:
irriigated_SF_data["survey_year"] = pd.to_datetime(irriigated_SF_data["LstSrvD"], 
                                                                format='mixed').dt.year

irriigated_SF_data.head(2)

,ID,CropTyp,Acres,ExctAcr,Irrigtn,LstSrvD,DataSrc,county,survey_year
0,51380_WSDA_SF_2016,bluegrass seed,6,6.116621,sprinkler,2016/09/19 00:00:00,wsda,Adams,2016
1,51381_WSDA_SF_2016,wheat,117,116.743101,center pivot,2016/09/20 00:00:00,wsda,Adams,2016


In [11]:
irriigated_SF_data.head(3)

,ID,CropTyp,Acres,ExctAcr,Irrigtn,LstSrvD,DataSrc,county,survey_year
0,51380_WSDA_SF_2016,bluegrass seed,6,6.116621,sprinkler,2016/09/19 00:00:00,wsda,Adams,2016
1,51381_WSDA_SF_2016,wheat,117,116.743101,center pivot,2016/09/20 00:00:00,wsda,Adams,2016
2,51383_WSDA_SF_2016,wheat,113,113.432725,center pivot,2016/09/13 00:00:00,wsda,Adams,2016


In [12]:
irriigated_SF_data["SF_year"] = irriigated_SF_data["ID"].str.split("_", expand=True)[3]
irriigated_SF_data["SF_year"] = irriigated_SF_data["SF_year"].astype(int)
irriigated_SF_data.head(2)

,ID,CropTyp,Acres,ExctAcr,Irrigtn,LstSrvD,DataSrc,county,survey_year,SF_year
0,51380_WSDA_SF_2016,bluegrass seed,6,6.116621,sprinkler,2016/09/19 00:00:00,wsda,Adams,2016,2016
1,51381_WSDA_SF_2016,wheat,117,116.743101,center pivot,2016/09/20 00:00:00,wsda,Adams,2016,2016


In [13]:
(irriigated_SF_data["SF_year"] == irriigated_SF_data["survey_year"]).sum()

44850

In [14]:
irriigated_SF_data.shape

(53582, 10)

In [15]:
# limit to large fields
# irriigated_SF_data = irriigated_SF_data[irriigated_SF_data.ExctAcr > 10]

In [16]:
irriigated_SF_data.head(2)

,ID,CropTyp,Acres,ExctAcr,Irrigtn,LstSrvD,DataSrc,county,survey_year,SF_year
0,51380_WSDA_SF_2016,bluegrass seed,6,6.116621,sprinkler,2016/09/19 00:00:00,wsda,Adams,2016,2016
1,51381_WSDA_SF_2016,wheat,117,116.743101,center pivot,2016/09/20 00:00:00,wsda,Adams,2016,2016


In [17]:
len(list(irriigated_SF_data.CropTyp.unique()))

115

In [18]:
ground_truth_labels.shape

(3160, 2)

In [19]:
all_SF_data = pd.read_csv(data_part_of_shapefile_dir + "all_SF_data_concatenated.csv")
all_SF_data.head(2)

,ID,CropGrp,CropTyp,Acres,Irrigtn,IntlSrD,LstSrvD,DataSrc,Notes,TRS,county,RtCrpTy,ExctAcr,Shp_Lng,Shap_Ar
0,51374_WSDA_SF_2016,Other,crp/conservation,7,none,2003/07/01 00:00:00,2016/09/13 00:00:00,wsda,NaN,T19R32E26,Adams,Wheat,6.813661,1006.574464,27573.906990
1,51375_WSDA_SF_2016,Other,crp/conservation,87,none,2003/07/01 00:00:00,2016/12/31 00:00:00,nass,NaN,T20R31E27,Adams,NaN,86.717035,2555.870586,350931.390797


In [20]:
all_SF_data.shape

(77530, 15)

In [21]:
ground_truth_labels = pd.merge(ground_truth_labels, all_SF_data, how="left", on="ID")
print (f"{ground_truth_labels.shape = }")
ground_truth_labels.head(2)

ground_truth_labels.shape = (3160, 16)


,ID,Vote,CropGrp,CropTyp,Acres,Irrigtn,IntlSrD,LstSrvD,DataSrc,Notes,TRS,county,RtCrpTy,ExctAcr,Shp_Lng,Shap_Ar
0,99837_WSDA_SF_2017,2,Cereal Grain,barley,73,center pivot,NaN,2017/10/10,wsda,NaN,NaN,Grant,NaN,73.250981,NaN,NaN
1,114615_WSDA_SF_2017,1,Cereal Grain,barley,31,center pivot,NaN,2017/10/10,wsda,NaN,NaN,Grant,NaN,30.993504,NaN,NaN


In [22]:
ground_truth_labels = ground_truth_labels[['ID', 'Vote', 'CropTyp',
                                           'Acres', 'Irrigtn', 'LstSrvD',
                                           'DataSrc', 'county']]

ground_truth_labels.shape

(3160, 8)

In [23]:
print (all_SF_data.county.unique())
print ()
print (all_SF_data.Irrigtn.unique())

['Adams' 'Benton' 'Franklin' 'Yakima' 'Grant' 'Walla Walla']

['none' 'sprinkler' 'center pivot' 'rill' 'flood' 'drip' 'wheel line'
 'unknown' 'drip/micro-sprinkler' 'big gun' 'center pivot/rill'
 'micro-sprinkler' 'center pivot/sprinkler' 'rill/sprinkler'
 'sprinkler/wheel line' 'center pivot/none' 'center pivot/wheel line'
 'center pivot/drip' 'drip/sprinkler' 'hand' 'big gun/sprinkler'
 'big gun/center pivot' 'hand/sprinkler' 'drip/wheel line'
 'drip/sprinkler/wheel line' 'rill/wheel line' 'drip/rill'
 'big gun/wheel line' 'rill/sprinkler/wheel line' 'drip/big gun'
 'center pivot/sprinkler/wheel line' 'none/rill' 'none/sprinkler']


In [24]:
GT_cropTypes = list(ground_truth_labels.CropTyp.unique())
irriigated_SF_data = irriigated_SF_data[irriigated_SF_data.CropTyp.isin(GT_cropTypes)].copy()

irriigated_SF_data.shape

(51833, 10)

In [25]:
irriigated_SF_data.ExctAcr.min()

0.500215389542

In [26]:
all_SF_data.CropTyp = all_SF_data.CropTyp.str.lower()

In [67]:
## see which crop types are included in training process
## where all fields of that crop are included (not just 10%).

crops_100Percent_in_train = []
percentage_dict = {}

# for crop in GT_cropTypes:
#     curr_irr_all = irriigated_SF_data[irriigated_SF_data.CropTyp == crop].copy()
#     GT_fields = ground_truth_labels[ground_truth_labels.CropTyp == crop].copy()
#     if len(curr_irr_all) == len(GT_fields):
#         crops_100Percent_in_train += [crop]
        
#     percentage_dict[crop] = round( (len(GT_fields) / len(curr_irr_all)) * 100, 2)
    
    


for crop in GT_cropTypes:
    curr_irr_all = irriigated_SF_data[irriigated_SF_data.CropTyp == crop].copy()
    curr_irr_all_large = curr_irr_all[curr_irr_all.ExctAcr > 10 ]
    GT_fields = ground_truth_labels[ground_truth_labels.CropTyp == crop].copy()
    
    perc_ = round( (len(GT_fields) / len(curr_irr_all)) * 100, 2)
    perc_large_ = round( (len(GT_fields) / len(curr_irr_all_large)) * 100, 2)
    percentage_dict[crop] = [perc_, perc_large_]
    

In [74]:
ground_truth_labels.shape

(3160, 8)

In [76]:
print (irriigated_SF_data[irriigated_SF_data.CropTyp == "canola"].shape)
print (irriigated_SF_data[(irriigated_SF_data.CropTyp == "canola") & \
                         (irriigated_SF_data.ExctAcr > 10)].shape)

print (ground_truth_labels[ground_truth_labels.CropTyp == "canola"].shape)


# IF we look at irriigated_SF_data There should be 6 fields of canola.
# why there are 27? Look at evaluation set. And why evaluation set is larger than
# irriigated_SF_data? counties?

(52, 10)
(49, 10)
(27, 8)


In [68]:
percentage_dict

{'barley': [34.38, 52.38],
 'barley hay': [23.53, 63.16],
 'bean, dry': [8.1, 10.14],
 'bean, green': [80.0, 95.65],
 'bluegrass seed': [17.36, 20.9],
 'buckwheat': [29.93, 39.42],
 'canola': [51.92, 55.1],
 'carrot': [29.41, 33.83],
 'corn, field': [6.95, 9.01],
 'corn, sweet': [6.75, 8.8],
 'grass hay': [4.0, 11.65],
 'grass seed': [33.01, 40.96],
 'market crops': [4.61, 12.5],
 'mint': [5.03, 6.58],
 'onion': [7.6, 9.57],
 'pea, green': [10.26, 12.31],
 'potato': [8.0, 9.46],
 'triticale': [33.33, 41.86],
 'triticale hay': [38.64, 70.83],
 'wheat': [6.46, 8.32],
 'wheat fallow': [26.09, 34.43],
 'yellow mustard': [51.28, 74.07],
 'alfalfa seed': [6.34, 7.82],
 'oat hay': [20.0, 55.56],
 'carrot seed': [37.78, 54.84],
 'pea, dry': [47.27, 60.47],
 'corn seed': [5.9, 28.57],
 'pea seed': [4.55, 8.33],
 'alfalfa hay': [0.77, 1.13],
 'apple': [6.1, 11.32],
 'apricot': [12.31, 53.33],
 'asparagus': [11.56, 21.9],
 'blueberry': [9.34, 17.78],
 'cherry': [3.91, 10.86],
 'fallow': [2.18, 7.

In [53]:
# "ryegrass seed" and "medicinal herb" 
ground_truth_labels[ground_truth_labels.CropTyp == "ryegrass seed"]

,ID,Vote,CropTyp,Acres,Irrigtn,LstSrvD,DataSrc,county
2593,101137_WSDA_SF_2018,1,ryegrass seed,62,center pivot,2018/10/09 00:00:00,wsda,Franklin
2594,107433_WSDA_SF_2018,1,ryegrass seed,32,center pivot,2018/10/09 00:00:00,wsda,Franklin
2876,100717_WSDA_SF_2018,1,ryegrass seed,41,center pivot,2018/10/09 00:00:00,wsda,Franklin
3030,102963_WSDA_SF_2018,1,ryegrass seed,87,center pivot/sprinkler,2018/10/05 00:00:00,wsda,Franklin
3031,99641_WSDA_SF_2018,1,ryegrass seed,147,center pivot,2018/08/31 00:00:00,wsda,Franklin
3135,101085_WSDA_SF_2018,1,ryegrass seed,35,center pivot,2018/10/09 00:00:00,wsda,Franklin
3136,102799_WSDA_SF_2018,1,ryegrass seed,38,center pivot/sprinkler,2018/10/08 00:00:00,wsda,Franklin
3137,105929_WSDA_SF_2018,1,ryegrass seed,72,center pivot,2018/10/08 00:00:00,wsda,Franklin


In [38]:
list(ground_truth_labels.CropTyp.unique())

['barley',
 'barley hay',
 'bean, dry',
 'bean, green',
 'bluegrass seed',
 'buckwheat',
 'canola',
 'carrot',
 'corn, field',
 'corn, sweet',
 'grass hay',
 'grass seed',
 'market crops',
 'mint',
 'onion',
 'pea, green',
 'potato',
 'triticale',
 'triticale hay',
 'wheat',
 'wheat fallow',
 'yellow mustard',
 'alfalfa seed',
 'oat hay',
 'carrot seed',
 'pea, dry',
 'corn seed',
 'pea seed',
 'alfalfa hay',
 'apple',
 'apricot',
 'asparagus',
 'blueberry',
 'cherry',
 'fallow',
 'fallow, idle',
 'fallow, tilled',
 'fescue seed',
 'grape, juice',
 'grape, wine',
 'hops',
 'medicinal herb',
 'mustard',
 'nectarine/peach',
 'onion seed',
 'orchard, unknown',
 'pasture',
 'pear',
 'pepper',
 'plum',
 'poplar',
 'pumpkin',
 'ryegrass seed',
 'sod farm',
 'squash',
 'sudangrass',
 'sugar beet',
 'sugar beet seed',
 'sunflower',
 'sunflower seed',
 'timothy',
 'unknown',
 'watermelon',
 'wildlife feed']

In [39]:
ground_truth_labels[ground_truth_labels.CropTyp == "wildlife feed"].shape

(27, 8)

In [43]:
irriigated_SF_data.head(2)

,ID,CropTyp,Acres,ExctAcr,Irrigtn,LstSrvD,DataSrc,county,survey_year,SF_year
0,51380_WSDA_SF_2016,bluegrass seed,6,6.116621,sprinkler,2016/09/19 00:00:00,wsda,Adams,2016,2016
1,51381_WSDA_SF_2016,wheat,117,116.743101,center pivot,2016/09/20 00:00:00,wsda,Adams,2016,2016


In [44]:
irriigated_SF_data[irriigated_SF_data.CropTyp == "barley hay"].shape

(51, 10)

In [45]:
ground_truth_labels[ground_truth_labels.CropTyp == "barley hay"].shape

(12, 8)

In [52]:
irriigated_SF_data[(irriigated_SF_data.CropTyp == "barley hay") &\
                   (irriigated_SF_data.ExctAcr > 10)].shape

(19, 10)